In [4]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 27.1 MB/s eta

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
from google.colab import files
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging
logging.basicConfig(level = logging.ERROR)
transformers_logger =  logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)
%env WAND_DISABLED = True



env: WAND_DISABLED=True


In [25]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
# plt.style.use('ggplot')

In [55]:
# Load the File
df_str_rel = pd.read_csv('/content/drive/MyDrive/Task A/eng/eng_train.csv', usecols=['PairID','Text','Score'])
df_str_rel.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [56]:
df_str_rel.describe()

,Score
count,5500.000000
mean,0.500109
std,0.221323
min,0.000000
25%,0.340000
50%,0.470000
75%,0.660000
max,1.000000


In [57]:
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel['input_text'] = [x[0] for x in df_str_rel['Split_Text']]
df_str_rel['target_text'] = [x[1] for x in df_str_rel['Split_Text']]
df_str_rel


,PairID,Text,Score,Split_Text,input_text,target_text
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"[It that happens, just pull the plug., if that...","It that happens, just pull the plug.","if that ever happens, just pull the plug."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,"[A black dog running through water., A black d...",A black dog running through water.,A black dog is running through some water.
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,"[I've been searchingthe entire abbey for you.,...",I've been searchingthe entire abbey for you.,I'm looking for you all over the abbey.
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,[If he is good looking and has a good personal...,If he is good looking and has a good personali...,"If he's good looking, and a good personality, ..."
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"[She does not hate you, she is just annoyed wi...","She does not hate you, she is just annoyed wit...","She doesn't hate you, she is just annoyed."
...,...,...,...,...,...,...
5495,ENG-train-5495,A young boy pounding on an anvil.\nWoman sits ...,0.0,"[A young boy pounding on an anvil., Woman sits...",A young boy pounding on an anvil.,Woman sits on the curb talking on a cellphone.
5496,ENG-train-5496,I love how he recognized his wife tempered his...,0.0,[I love how he recognized his wife tempered hi...,I love how he recognized his wife tempered his...,"Torpedo Ink is Viktor's Band of Brothers, the ..."
5497,ENG-train-5497,I actually read a chapter or two beyond that p...,0.0,[I actually read a chapter or two beyond that ...,I actually read a chapter or two beyond that p...,Lets say she's a blend of two types of beings.
5498,ENG-train-5498,A boy gives being in the snow two thumbs up.\n...,0.0,"[A boy gives being in the snow two thumbs up.,...",A boy gives being in the snow two thumbs up.,A satisfied cat is perched beside a crystal lamp.


In [58]:
df_str_rel["prefix"] = "generate paraphrase"
df_str_rel = df_str_rel.rename(columns={"sentence 1":"input_text", "sentence 2":"target_text"})

In [64]:
df = pd.read_csv("/content/drive/MyDrive/Task A/eng/eng_dev.csv", usecols=['PairID','Text'])


In [65]:
df_str_rel.describe()

,Score
count,5500.000000
mean,0.500109
std,0.221323
min,0.000000
25%,0.340000
50%,0.470000
75%,0.660000
max,1.000000


In [66]:
df['Split-text'] = df['Text'].apply(lambda x: x.split("\n"))
df['input_text'] = [x[0] for x in df['Split-text']]
df['target_text'] = [x[1] for x in df['Split-text']]
df

,PairID,Text,Split-text,input_text,target_text
0,ENG-dev-0000,The story is gripping and interesting.\nIt's a...,"[The story is gripping and interesting., It's ...",The story is gripping and interesting.,"It's a brilliant, compelling, and heartfelt st..."
1,ENG-dev-0001,The majority of Southeast Alaska 's area is pa...,[The majority of Southeast Alaska 's area is p...,The majority of Southeast Alaska 's area is pa...,A lot of of the panhandle is part of the Tonga...
2,ENG-dev-0002,and from your post i think you are to young to...,[and from your post i think you are to young t...,and from your post i think you are to young to...,I think it will be very bad if he acquires her...
3,ENG-dev-0003,The film 's success also made Dreamworks Anima...,[The film 's success also made Dreamworks Anim...,The film 's success also made Dreamworks Anima...,There have also been two sequels -LRB- follow-...
4,ENG-dev-0004,I am still confused about how I feel about thi...,[I am still confused about how I feel about th...,I am still confused about how I feel about thi...,"In this particular book, Blue and Gansey are s..."
...,...,...,...,...,...
245,ENG-dev-0245,thats just how they are :( its a shame lol\nIt...,"[thats just how they are :( its a shame lol, I...",thats just how they are :( its a shame lol,It is a shame that they are like that.
246,ENG-dev-0246,I feel sorry for the books that I will read af...,[I feel sorry for the books that I will read a...,I feel sorry for the books that I will read af...,: More than one person recommended this book t...
247,ENG-dev-0247,Uwe Seeler -LRB- born 5 November 1936 in Hambu...,[Uwe Seeler -LRB- born 5 November 1936 in Hamb...,Uwe Seeler -LRB- born 5 November 1936 in Hambu...,Alain Giresse -LRB- born 2 August 1952 in Lang...
248,ENG-dev-0248,Waco is a city in and the county seat of McLen...,[Waco is a city in and the county seat of McLe...,Waco is a city in and the county seat of McLen...,It is the county seat of Morris County .


In [67]:
df["prefix"] = "generate paraphrase"
df = df.rename(columns = {"sentence 1":"input_text","sentence 2":"target_text"})

In [82]:
model_args ={
    "reprocess_input_data":True,
    "overwrite_output_dir":True,
    "max_seq_length":128,
    "train_batch_size":16,
    "num_train_epochs":20,
    "num_beams":None,
    "do_sample":True,
    "max_length":20,
    "top_k":50,
    "top_p":0.95,
    "use_multiprocessing":False,
    "save_steps":-1,
    "save_eval_checkpoints":True,
    "evaluate_during_training":True,
    "evaluate_during_training_verbose":True,
    "num_return_sequences":5,

}

In [88]:
from simpletransformers.t5 import T5Model

In [89]:
model = T5Model("t5","t5-small", args = model_args)
model.train_model(df_str_rel,eval_data=df)

  0%|          | 0/5500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/344 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Running Epoch 19 of 20:   0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


(6880,
 {'global_step': [344,
   688,
   1032,
   1376,
   1720,
   2000,
   2064,
   2408,
   2752,
   3096,
   3440,
   3784,
   4000,
   4128,
   4472,
   4816,
   5160,
   5504,
   5848,
   6000,
   6192,
   6536,
   6880],
  'eval_loss': [2.851577727124095,
   2.901331301778555,
   2.9273270070552826,
   3.034458789974451,
   3.136576797813177,
   3.2471355609595776,
   3.268679451197386,
   3.3999617844820023,
   3.507647752761841,
   3.627060614526272,
   3.751562677323818,
   3.9604333452880383,
   4.004954047501087,
   4.047350935637951,
   4.225156791508198,
   4.3589121252298355,
   4.483624920248985,
   4.7506226524710655,
   4.845755353569984,
   5.045758374035358,
   5.049343094229698,
   5.099340416491032,
   5.20276415348053],
  'train_loss': [3.1502342224121094,
   3.1849160194396973,
   2.8902454376220703,
   2.2814512252807617,
   2.4998764991760254,
   2.114553689956665,
   2.2401232719421387,
   1.832431435585022,
   2.001652240753174,
   1.2407026290893555,
   1.6

In [90]:
import os

# Define the directory path
directory_path = '/content/gdrive/My Drive/T5'

# Use os.makedirs to create the directory and its parent directories if they don't exist
os.makedirs(directory_path, exist_ok=True)


In [91]:
!cp -r /content/outputs/best_model/ /content/gdrive/'My Drive'/T5/

In [92]:
# df_str_rel.columns

In [93]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = "/content/gdrive/My Drive/T5/best_model"  # Replace with the correct path to your model

model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

In [96]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer


input_text = "generate paraphrase:the last NBA season was amazing."
input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True)
print("Original sentence",input_text)
for x in range(3):
    # Generate a single paraphrase
    generated = model.generate(input_ids, max_length=100, num_return_sequences=1)

    # Convert the generated output back to text
    generated_paraphrase = tokenizer.decode(generated[0], skip_special_tokens=True)
    print("Generated Paraphrase:", generated_paraphrase)


Original sentence generate paraphrase:the last NBA season was amazing.
Generated Paraphrase: The last NBA season was amazing.
Generated Paraphrase: The last NBA season was amazing.
Generated Paraphrase: The last NBA season was amazing.
